In [4]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

In [25]:
torch.cuda.is_available()

False

In [7]:
torch.cpu.device_count()

1

In [8]:
DEVICE = torch.device("cpu")

### Загрузка Сбер модели

In [9]:
sber_model_path = "sberbank-ai/rugpt3large_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(sber_model_path)
model = GPT2LMHeadModel.from_pretrained(sber_model_path).to(DEVICE)

c:\Users\ipuzanov\PycharmProjects\BMSTU\venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Проверка работы модели на математическом примере

In [23]:
text = " Вопрос: 'Сколько будет 3+3?' \n Ответ: 6 . \n Вопрос: 'Сколько будет 1+9?' \n Ответ: 10 . \n Вопрос: 'Сколько будет 4+2?' \n Ответ:"
input_tokens = tokenizer.encode(text, return_tensors="pt").to(DEVICE)

In [24]:
print(f'{input_tokens}\nКоличество токенов: {len(input_tokens[0])}')

tensor([[ 8919,    30,  6145, 38687,   840,   816,    15,    23,    35,    11,
           372,  8353,    30,  1286,  1492,   372,  8919,    30,  6145, 38687,
           840,   430,    15,    29,    35,    11,   372,  8353,    30,  1533,
          1492,   372,  8919,    30,  6145, 38687,   840,  1032,    15,    22,
            35,    11,   372,  8353,    30]])
Количество токенов: 45


In [20]:
out = model.generate(input_tokens, do_sample=False, max_length=50)

In [21]:
generated_text = list(map(tokenizer.decode, out))[0]
print(generated_text)

 Вопрос: 'Сколько будет 3+3?' 
 Ответ: 6. 
 Вопрос: 'Сколько будет 1+9?' 
 Ответ: 10. 
 Вопрос: 'Сколько будет 4+2?' 
 Ответ: 12. 
 Вопрос:


### Пробуем продолжить текст

In [32]:
original_text = """
    Так, в конторе губернской тюрьмы считалось священным и важным не то, 
    что всем животным и людям даны умиление и радость весны, 
    а считалось священным и важным то, что накануне получена была за номером с печатью и заголовком бумага о том, 
    чтобы к девяти часам утра были доставлены в нынешний день, 
    28-го апреля, три содержащиеся в тюрьме подследственные арестанта — две женщины и один мужчина.
"""

In [45]:
len(tokenizer.encode(original_text, return_tensors="pt").to(DEVICE)[0])

96

In [47]:
MAX_LENGTH = 80

In [29]:
text = "Так, в конторе губернской тюрьмы считалось"

In [33]:
input_tokens = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
print(f'{input_tokens}\nКоличество токенов: {len(input_tokens[0])}')

tensor([[ 5673,    16,   282, 48065, 11155, 35228, 16886, 25399]])
Количество токенов: 8


### Используем Argmax. Модель берет наиболее подходящее слово. Результат раз от раза не меняется

In [48]:
out = model.generate(input_tokens, do_sample=False, max_length=MAX_LENGTH)
print(f'Количество токенов: {len(out[0])}')

Количество токенов: 80


In [49]:
generated_text = list(map(tokenizer.decode, out))[0]
print(generated_text)

Так, в конторе губернской тюрьмы считалось, что в случае побега заключенные должны были быть расстреляны.

В конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов,


In [50]:
out_2 = model.generate(input_tokens, do_sample=False, max_length=MAX_LENGTH)
print(f'Количество токенов: {len(out_2[0])}')

Количество токенов: 80


In [51]:
generated_text_2 = list(map(tokenizer.decode, out_2))[0]
print(generated_text_2)

Так, в конторе губернской тюрьмы считалось, что в случае побега заключенные должны были быть расстреляны.

В конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов,


видим, что результат не меняется

### Включим сэмплирование (do_sample). Модель выбирает слово из N наиболее подходящих

In [52]:
out = model.generate(input_tokens, do_sample=True, top_k=0, max_length=MAX_LENGTH)
print(f'Количество токенов: {len(out[0])}')

Количество токенов: 80


In [53]:
generated_text = list(map(tokenizer.decode, out))[0]
print(generated_text)

Так, в конторе губернской тюрьмы считалось, что Александр Митрофанов умер «от сильных побоев». За его тело была назначена сумма в 13 тысяч рублей.

В докладе Гатчинского полицмейстера от 24 марта 1906 г. Шотландский лейб-гвардии полк, находился "сожалении" о том, что в последние двое суток обстановка на допросных пунктах от внутри


In [54]:
out_2 = model.generate(input_tokens, do_sample=True, top_k=0, max_length=MAX_LENGTH)
print(f'Количество токенов: {len(out_2[0])}')

Количество токенов: 80


In [55]:
generated_text_2 = list(map(tokenizer.decode, out_2))[0]
print(generated_text_2)

Так, в конторе губернской тюрьмы считалось очень почетным бывать сынам опричнины. Они сюда проникали разными способами и являлись сюда затем, чтобы вести воспитанные с помощью криминальных методов, беседы.

Предчувствие тяготило, даже трепетало всю предыдущую неделю, и Андрей стал нервничать, ожидая возвращения к себе в камеру. Но строгие инструкции его начальника, инструкция нев


Видим, что теперь результаты разные

### Используем параметр top_k. При его увеличении, модель будет выбирать из большего количетсва вариантов. По умолчанию используется 50. Попробуем разные варианты этого параметра

In [56]:
for k in (1, 3, 5, 8, 50, 100):
    out = model.generate(input_tokens, do_sample=True, top_k=k, max_length=MAX_LENGTH)
    generated_text = list(map(tokenizer.decode, out))[0]

    print(f'Текст для k={k}:')
    print(generated_text, end='\n\n')

Текст для k=1:
Так, в конторе губернской тюрьмы считалось, что в случае побега заключенные должны были быть расстреляны.

В конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов,

Текст для k=3:
Так, в конторе губернской тюрьмы считалось, что он был арестован по делу о краже. Но в действительности он был арестован по делу о краже. В тюрьме он был арестован по обвинению в убийстве, а затем, в результате недоразумения, был освобожден. В результате он был освобожден, но не в том виде, который был ему нужен. В тюрьме он был освобожден, но не

Текст для k=5:
Так, в конторе губернской тюрьмы считалось, что, если бы не было такой сильной и многочисленной охраны, как в Петербурге, то, вероятно, заключенные были бы убиты или замучены до смерти.
В 1832 году в Петербурге, по распоряжению генерал-прокурора князя Вяземского, была учре

Видим, что небольшое значение k идентично argmax, т.е. происходит зацикливание. С увеличением кол-ва вариантов растет и разнообразие текста

### Использование температуры. Температура позволяет регулировать распределение. При высоком значении распределение сглаживается, при низких заостряется

In [60]:
for t in (0.01, 0.1, 1., 2., 3., 5.):
    out = model.generate(input_tokens, do_sample=True, top_k=0, temperature=t, max_length=MAX_LENGTH)
    generated_text = list(map(tokenizer.decode, out))[0]

    print(f'Текст для temperature={t}:')
    print(generated_text, end='\n\n\n')

Текст для temperature=0.01:
Так, в конторе губернской тюрьмы считалось, что в случае побега заключенные должны были быть расстреляны.

В конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов,


Текст для temperature=0.1:
Так, в конторе губернской тюрьмы считалось, что в случае побега заключенные должны были быть расстреляны.

В конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов, в конце концов,


Текст для temperature=1.0:
Так, в конторе губернской тюрьмы считалось, что марксист-большевик затратил «немалые средства на своего рода упрощение нравов» заключенных.

О том, какие заработки давала маковка Рижской Преображенской тюрьмы, нашему земскому работнику сведения были по

Аналогично с предыдущим параметром, при низкой температуре происходит зацикливание. Но при температуре выше 1 появляется бессмысленный текст

### Использование Beam Search. Данных механизм добавляет генерацию нескольких текстов параллельно и выбор на каждом шаге наиболее подходящих. Параметр num_beams настраивает количество параллельных вариантов

In [64]:
for n in (1, 3, 5, 8):
    out = model.generate(input_tokens, do_sample=True, top_k=0, num_beams=n, max_length=MAX_LENGTH)
    generated_text = list(map(tokenizer.decode, out))[0]

    print(f'Текст для num beams={n}:')
    print(generated_text, end='\n\n\n')

Текст для num beams=1:
Так, в конторе губернской тюрьмы считалось, что во времена нашего вождя Н. С. Хрущева отношения между харьковскими чекистами и особые отделы НКВД были вполне дружеские — каждую неделю устраивали дружеские вечеринки и дни рождения чекистов, печенеги и казаки в город приезжали, мордобоя не было — каюк был работе штата вверенных им спецслужбишек, вобщем


Текст для num beams=3:
Так, в конторе губернской тюрьмы считалось, что если заключенный освобожден по амнистии, то он не может быть привлечен к уголовной ответственности.

В 20-е годы были приняты особые законы, ограничивающие применение помилования.

По новому закону, изданному в 1921 году, амнистия не могла применяться к лицам, совершившим тяжкие преступления, а также к лицам, соверши


Текст для num beams=5:
Так, в конторе губернской тюрьмы считалось, что в случае побега арестанта из тюрьмы он должен быть немедленно схвачен и доставлен в губернскую жандармерию. На деле арестант мог свободно гулять по городу, ход

Увеличение количетсва beams привело к улучшению попадания в тему (но так не всегда, зависит от прогона)

### Попробуем использовать все исследованные параметры

In [69]:
out = model.generate(input_tokens, do_sample=True, top_k=5, temperature=0.8, num_beams=7, max_length=MAX_LENGTH)
print(f'Количество токенов: {len(out[0])}')

Количество токенов: 80


In [70]:
generated_text = list(map(tokenizer.decode, out))[0]
print(generated_text)

Так, в конторе губернской тюрьмы считалось, что в случае побега арестанта из-под стражи он должен быть немедленно доставлен в губернскую жандармерию.

— А что, если я сбегу? — спросил я.

— Сбежишь?

— Да.

— Сбежишь?

— Да.




Совместное использование параметров не помогло :) Получилось зацикливание

In [72]:
out = model.generate(input_tokens, do_sample=True, top_k=5, temperature=0.8, max_length=MAX_LENGTH)
generated_text = list(map(tokenizer.decode, out))[0]
print(generated_text)

Так, в конторе губернской тюрьмы считалось, что он, будучи в ссылке и не имея возможности работать по профессии, может только писать и читать. Но и в этой профессии он, как и прежде, не нашел себе применения.

В конце 1872 года, когда ему уже исполнилось шестьдесят два года, он снова приехал в Россию.

В Петербурге он поступил на службу в канцелярию губернатора.


Если убрать beams, то получается лучше